In [1]:
import numpy as np
import pandas as pd
import sys
from openai import OpenAI
import os
import cv2
import re
import glob
from pathlib import Path
import matplotlib.pyplot as plt
import base64
from tqdm import tqdm
from dotenv import load_dotenv
import re
from google import genai

In [2]:
df = pd.read_csv('Data/New230SHEETS.csv')
df

,id,image_name,ocr_txt
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...
...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n..."
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah..."
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...


In [3]:
# Check for missing values in the 'image_name' column
missing_image_names = df['image_name'].isna()

# Show rows where 'image_name' is missing
df_missing = df[missing_image_names]
print(df_missing)


Empty DataFrame
Columns: [id, image_name, ocr_txt]
Index: []


In [4]:
# Drop rows where 'image_name' is missing
df = df.dropna(subset=['ocr_txt'])

# Optionally, reset the index if you want a clean index after dropping
df = df.reset_index(drop=True)

In [5]:
# Load prompt template
with open('Prompt/interpreter.md', 'r', encoding='utf-8') as f:
    prompt_template = f.read()

In [6]:
# Test with first meme
test_row = df.iloc[0]
image_text = test_row['ocr_txt']
image_name = test_row['image_name']

# Replace placeholders
final_prompt = prompt_template.replace('{meme_text}', str(image_text))
final_prompt = final_prompt.replace('{image_path}', image_name)

# Show the final prompt
print("FINAL PROMPT THAT WILL BE SENT TO OPENAI:")
print("="*60)
print(final_prompt)
print("="*60)

FINAL PROMPT THAT WILL BE SENT TO OPENAI:
# Role
You are an expert in interpreting meme since you are a master of analyzing the hidden message behind meme in Indonesian language

# Task
Your task is to generate a hidden message or the context of the meme

## Things Need to be Considered
- There are 2 inputs which are meme image and meme text. Please, refer to those input for interpret meme
- All of the memes are in Indonesian language. Therefore, please interpret meme in Indonesian language
- The connection between meme image and meme text can be metaporical. Therefore, analyze both of those input carefully (i.e., a picture of a grumpy man is represent bad stuff and dislike)

## Example

### Example 1
Input Image = meme_data_999.jpg
Input Text = Sudah Kubilang jangan lawan King Indonesia
Output = Tim sepak bola Indonesia adalah yang terbaik saat ini dan susah untuk dikalahkan. Oleh karena itu jangan coba-coba melawan tim sepak bola Indonesia

## Query
Input Image = Meme_Data_1.jpg
Inpu

In [7]:
def encode_image_to_base64(image_path):
    """Encode image to base64"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [11]:
# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from environment
client = OpenAI(
    api_key=os.getenv('GEMINI_API_KEY'), base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Verify API key is loaded
if not os.getenv('GEMINI_API_KEY'):
    raise ValueError("GEMINI_API_KEY not found in environment variables. Please create a .env file with your API key.")

def interpret_meme(row):
    """Interpret meme using OpenAI Vision API"""
    # Get image path and text from your CSV structure
    image_path = f"Data/Filtered 230/{row['image_name']}"
    image_text = row['ocr_txt']
    
    # Encode image
    base64_image = encode_image_to_base64(image_path)
    
    # Create prompt by replacing placeholders
    prompt = prompt_template.replace('{meme_text}', str(image_text))
    prompt = prompt.replace('{image_path}', row['image_name'])
    
    try:
        response = client.chat.completions.create(
            model="gemini-2.5-pro",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            max_tokens=2048,
            temperature=0
        )

        result = response.choices[0].message.content
        return result
    except Exception as e:
        print("Error:", e)
        return None

In [36]:
sample_row = df.iloc[4]
sample_row

id                                                            5
image_name                                      Meme_Data_5.jpg
ocr_txt       KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...
Name: 4, dtype: object

In [37]:
sample_row = df.iloc[4]  # Use the first row for testing
output = interpret_meme(sample_row)
print(output)

Tentu, ini adalah interpretasi dari meme tersebut:

Meme ini adalah sebuah sindiran politik yang ditujukan kepada salah satu calon presiden di Indonesia, yaitu Prabowo Subianto. Meme ini menggunakan format "We Are Not The Same" (Kita Tidak Sama) yang dipopulerkan oleh karakter Gus Fring dari serial TV *Breaking Bad* untuk menunjukkan superioritas alasan seseorang dibandingkan orang lain.

Berikut adalah pembedahan pesan di dalam meme:

1.  **"KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FANATISME CALON"**: Bagian ini merujuk pada alasan umum atau alasan yang dianggap dangkal mengapa seseorang tidak memilih seorang calon. Alasan seperti "beda aliran" (berbeda pandangan politik/


In [38]:
output

'Tentu, ini adalah interpretasi dari meme tersebut:\n\nMeme ini adalah sebuah sindiran politik yang ditujukan kepada salah satu calon presiden di Indonesia, yaitu Prabowo Subianto. Meme ini menggunakan format "We Are Not The Same" (Kita Tidak Sama) yang dipopulerkan oleh karakter Gus Fring dari serial TV *Breaking Bad* untuk menunjukkan superioritas alasan seseorang dibandingkan orang lain.\n\nBerikut adalah pembedahan pesan di dalam meme:\n\n1.  **"KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FANATISME CALON"**: Bagian ini merujuk pada alasan umum atau alasan yang dianggap dangkal mengapa seseorang tidak memilih seorang calon. Alasan seperti "beda aliran" (berbeda pandangan politik/'

In [14]:
# Process all memes
results = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing memes"):
    result = interpret_meme(row)
    results.append({
        'id': row['id'],
        'image_name': row['image_name'],
        'ocr_txt': row['ocr_txt'],
        'full_response': result
    })
    
    # Optional: Add delay to respect rate limits
    import time
    time.sleep(1)

results

Processing memes:  92%|█████████▏| 211/230 [1:28:35<06:35, 20.79s/it]

Error: Connection error.


Processing memes: 100%|██████████| 230/230 [1:36:59<00:00, 25.30s/it]


[{'id': 1,
  'image_name': 'Meme_Data_1.jpg',
  'ocr_txt': 'COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK GILIRAN DIMINTAIN DUIT SALON, BILANGNYA MATRE EMANG LO KIRA PERAWATAN DI SALON PAKE BPJS??',
  'full_response': None},
 {'id': 2,
  'image_name': 'Meme_Data_2.jpg',
  'ocr_txt': 'ayat suci dalam bio bukan jaminan kalo\n dia orang yang baik',
  'full_response': 'Meme ini menyampaikan pesan bahwa seseorang yang mencantumkan kutipan dari kitab suci di bio media sosialnya tidak serta merta mencerminkan bahwa orang tersebut memiliki kepribadian atau perilaku yang baik. Ini adalah sebuah sindiran bahwa penampilan religius di dunia maya, seperti memasang ayat suci, bukanlah jaminan karakter asli seseorang. Kebaikan sejati seseorang dinilai dari tindakan dan perilakunya sehari-hari, bukan dari citra yang ingin ia tampilkan di media sosial.'},
 {'id': 3,
  'image_name': 'Meme_Data_3.jpg',
  'ocr_txt': 'Rare\n Medium Rare\n Medium\n Medium Well\n Well Done\n ONDE MANDE',
  'full_response': 

In [22]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df

,id,image_name,ocr_txt,full_response
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,None
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa seseorang ya...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini secara humoris membandingkan tingkat ...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menyampaikan pesan kesombongan yang d...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,None
...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Meme ini menyindir fenomena sosial di mana ora...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Meme ini menggambarkan perasaan frustrasi dan ...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyajikan perbandingan ironis antara...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Meme ini menceritakan tentang seorang siswa be...


In [ ]:
def extract_combined_robust(response):
    # Try to extract after 'output ='
    match_output = re.search(r'output\s*=\s*([\s\S]*)', response, re.IGNORECASE)
    if match_output:
        return match_output.group(1).strip()
    # Try to extract after 'Pesan tersembunyi dari meme ini adalah:'
    match_pesan = re.search(r'Tentu, ini adalah interpretasi dari meme tersebut:\s*([\s\S]*)', response, re.IGNORECASE)
    if match_pesan:
        return match_pesan.group(1).strip()
    # Fallback to original response
    return response.strip()

results_df['output_extracted'] = results_df['full_response'].apply(lambda x: extract_combined_robust(str(x)))

In [ ]:
results_df['output_extracted'] = results_df['output_extracted'].str.replace(
    r"\*\*Konteks atau Pesan Tersembunyi:\*\*\s*\n?", "", regex=True
)

results_df['output_extracted'] = results_df['output_extracted'].str.replace(
    r"\*\*Pesan tersembunyi atau konteks dari meme ini adalah:\*\*\s*\n?", "", regex=True
)

In [25]:
# Example usage:
response = results_df['output_extracted'][141]
print(extract_combined_robust(response))

Meme ini membandingkan dua tipe pria dalam menyikapi pasangan yang tidak bisa memasak. Pria yang hanya bisa protes dan menuntut pasangannya untuk bisa memasak digambarkan sebagai sosok yang lemah dan tidak menarik. Sebaliknya, pria yang bisa memasak sendiri dianggap sebagai sosok yang superior, mandiri, dan keren (digambarkan dengan sosok GigaChad) karena ia tidak bergantung pada pasangannya dan tidak mempermasalahkan stereotip gender.


In [26]:
results_df

,id,image_name,ocr_txt,full_response,output_extracted
0,1,Meme_Data_1.jpg,COWOK JAMAN SEKARANG PENGEN PUNYA PACAR CANTIK...,None,None
1,2,Meme_Data_2.jpg,ayat suci dalam bio bukan jaminan kalo\n dia o...,Meme ini menyampaikan pesan bahwa seseorang ya...,Meme ini menyampaikan pesan bahwa seseorang ya...
2,3,Meme_Data_3.jpg,Rare\n Medium Rare\n Medium\n Medium Well\n We...,Meme ini secara humoris membandingkan tingkat ...,Meme ini secara humoris membandingkan tingkat ...
3,4,Meme_Data_4.jpg,KITA MAH ORANG SUSAH\n INWE\n SUSAH DILAWAN MA...,Meme ini menyampaikan pesan kesombongan yang d...,Meme ini menyampaikan pesan kesombongan yang d...
4,5,Meme_Data_5.jpg,KALIAN GA PILIH DIA KARENA BEDA ALIRAN DAN FAN...,None,None
...,...,...,...,...,...
225,226,Meme_Data_226.jpg,": ""Kamu dari jawa ya!"" Iyaa, kamu jawa juga?\n...",Meme ini menyindir fenomena sosial di mana ora...,Meme ini menyindir fenomena sosial di mana ora...
226,227,Meme_Data_227.jpg,"Pas kecil liat orang dewasa pacaran, pas sudah...",Meme ini menggambarkan perasaan frustrasi dan ...,Meme ini menggambarkan perasaan frustrasi dan ...
227,228,Meme_Data_228.jpg,SINGAPORE AIRLINES\n 1. Untung 47 Trilliun 2. ...,Meme ini menyajikan perbandingan ironis antara...,Meme ini menyajikan perbandingan ironis antara...
228,229,Meme_Data_229.jpg,Yudi\n \n P\n \n Bls\n \n Besok pake baju apa\...,Meme ini menceritakan tentang seorang siswa be...,Meme ini menceritakan tentang seorang siswa be...


In [ ]:
# Save results
results_df.to_csv('gemini_2.5_pro_results.csv', index=False)